### Lotame + LiveRamp + AIQ - File Creation


In [11]:
import pandas as pd
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

### Audience Dataframe Creation

In [12]:
audiences = {
    'Country': ['AU', 'AU', 'AU', 'BR', 'BR', 'BR', 'CA', 'CA', 'CA', 'FR', 'FR'],
    'Number': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2],
    'Provider': ['Lotame', 'AIQ', 'LR', 'Lotame', 'AIQ', 'LR', 'Lotame', 'AIQ', 'LR', 'Lotame', 'AIQ'],
    'Audience ID': ['dgfrrpdw7ov', 'd2q11zclsgv', 'duxo79h5t8o', 'dkgud3s0umw', 'dwm57ismy2g', 'd3ok01g6usv',
             'dv7sm9mltvm', 'd7qd2wu3gkb', 'dw1v0i89ctr', 'd6m18ot8p0w', 'd14s5u5pfra']
}

df_audiences = pd.DataFrame(audiences)

df_audiences


,Country,Number,Provider,Audience ID
0,AU,1,Lotame,dgfrrpdw7ov
1,AU,2,AIQ,d2q11zclsgv
2,AU,3,LR,duxo79h5t8o
3,BR,1,Lotame,dkgud3s0umw
4,BR,2,AIQ,dwm57ismy2g
5,BR,3,LR,d3ok01g6usv
6,CA,1,Lotame,dv7sm9mltvm
7,CA,2,AIQ,d7qd2wu3gkb
8,CA,3,LR,dw1v0i89ctr
9,FR,1,Lotame,d6m18ot8p0w


In [13]:
audience_codes = '(dgfrrpdw7ov|d2q11zclsgv|duxo79h5t8o|dkgud3s0umw|dwm57ismy2g|d3ok01g6usv|dv7sm9mltvm|d7qd2wu3gkb|dw1v0i89ctr|d6m18ot8p0w|d14s5u5pfra)'


### GCM Data Import - Impressions & Clicks

In [14]:
df_impressions = pd.read_csv(r'C:\Users\Rafael_Fagundes\Downloads\Impressions.csv', skiprows=10,skipfooter=1, parse_dates=['Date'])

In [15]:
# Filter by Audience IDs
df_impressions_filtered = df_impressions[df_impressions['Placement'].str.contains(audience_codes, case=False, regex=True)]

# Convert to datetime for the merge
df_impressions_filtered['Date'] = pd.to_datetime(df_impressions_filtered['Date'])

# Remove any '-'
df_impressions_filtered.replace('-', 0, inplace=True)

# Change Reach and Frequency datatypes for the sum
df_impressions_filtered['Impressions'] = df_impressions_filtered['Impressions'].astype(float)
df_impressions_filtered['Clicks'] = df_impressions_filtered['Clicks'].astype(float)

# Group by Date and Placement and sum Reach and Frequency
df_impressions_group = df_impressions_filtered.groupby(['Date', 'Placement'])[['Impressions', 'Clicks']].sum().reset_index()

### GCM Data Import - Reach & Frequency


In [16]:
df_reach = pd.read_csv(r'C:\Users\Rafael_Fagundes\Downloads\Reach.csv', skiprows=10,skipfooter=1, parse_dates=['Date'])

In [17]:
# Filter by Audience IDs
df_reach_filtered = df_reach[df_reach['Placement'].str.contains(audience_codes, case=False, regex=True)]

# Convert to datetime for the merge
df_reach_filtered['Date'] = pd.to_datetime(df_reach_filtered['Date'])

# Remove any '-'
df_reach_filtered.replace('-', 0, inplace=True)

# Rename columns: Reach and Frequency
df_reach_filtered.rename(columns={'Unique Reach: Impression Reach (Co-Viewed)': 'Reach', 'Unique Reach: Average Impression Frequency (Co-Viewed)': 'Frequency'}, inplace=True)

# Change Reach and Frequency datatypes for the sum
df_reach_filtered['Reach'] = df_reach_filtered['Reach'].astype(float)
df_reach_filtered['Frequency'] = df_reach_filtered['Frequency'].astype(float)

# Group by Date and Placement and sum Reach and Frequency
df_reach_group = df_reach_filtered.groupby(['Date', 'Placement'])[['Reach', 'Frequency']].sum().reset_index()


### Merge Audience Dataframe with GCM File

In [18]:
df_merge = pd.merge(df_impressions_group, df_reach_group, on=['Date','Placement'], how='left')

df_merge['Audience ID'] = df_merge['Placement'].str.lower().str.extract(audience_codes)

df_final = pd.merge(df_merge, df_audiences, on=['Audience ID'], how='left')

### Save Final File

In [20]:
df_final.to_csv(r'csv\df.csv', index=False)